In [4]:
import pandas as pd
import numpy as np
import re


In [8]:
data = pd.read_parquet("./data/final_products.parquet")
data.iloc[:10000].sample(25)

,SKU,ProdName,Price,SpecPrice,URL,PCatName,PGrName,PGrName1,PGrName2,product_descr_ukr,way_of_use_ukr
2647,1019409,"Пенал Bob Snail из полотна, розовый, 18*10 см",259.0,195.0,https://eva.ua/pr1360771/,Канцтовары,Школьные аксессуары,Шкільна канцелярія,Равлик боб (іграшки),Зберігайте все необхідне для малювання в одном...,
2213,1017447,Подарочный набор для бровей Mayur Уход и фикса...,314.0,266.0,https://eva.ua/pr1289840/,Косметика декоративная,Декоративная косметика для глаз и бровей,Специальные средства по уходу за бровям и ресн...,Маюр,Виразні та красиві брови навіть без косметики ...,Гель для брів: гель наносять на брови за допом...
3097,1021677,Протеин BioTech USA Beef Protein Ваниль и кори...,1065.0,920.0,https://eva.ua/pr1293266/#/173834/,"Диета, питание и витамины",Биологически активные добавки,Средства для контроля массы тела,БиоТек ЮСА,Яловичий протеїн є дуже популярним серед важко...,Рекомендоване дозування на одну порцію&nbsp;&m...
8129,1047796,Мужское масло для бороды и усов Nishman Beard ...,329.0,329.0,https://eva.ua/pr1351855/,Уход за волосами,Средства для мытья волос мужские,Лечебно-профилактические средства для ухода за...,Нишман,Nishman Beard &amp; Moustache Care Oil Gold On...,Нанесіть кілька крапель олії на долоні та рівн...
9315,1054501,Креатин Olimp Sport Nutrition Creatine 1250 Me...,1875.0,1786.0,https://eva.ua/pr1646285/#/79020/,"Диета, питание и витамины",Биологически активные добавки,Средства для контроля массы тела,Олимп Спорт Нутришн,Інтенсивні фізичні навантаження можуть виснажу...,Приймати 4 капсули на день після тренування аб...
1888,1016787,Успокаивающая сыворотка для лица Tocobo Cica C...,31.0,28.0,https://eva.ua/pr1260005/,Уход за лицом и телом,Кремы для лица,Сыворотки для лица,Токобо,Для комфорту і свіжості шкіри &mdash; сироватк...,Наносьте кілька крапель сироватки на очищене о...
8803,1052620,Lattafa Perfumes Pride La African Drummer Парф...,1999.0,1729.0,https://eva.ua/pr1366129/,Парфюмерия,Парфюмерия МидлМаркет_ММ,Парфюмерия унисекс Мидл маркет,ПММ Латтафа Парфюмс,Історія\r\n\r\nPride La African Drummer &mdash...,"Нанесіть аромат на зону шиї, ділянки за вухами..."
9159,1053941,"Подгузники Merries для новорожденных, размер N...",1550.0,1550.0,https://eva.ua/pr1381390/,Пеленание и гигиена малышей,Пеленание,Подгузники джамбо пачка,Меррис (КАО),Підгузки Merries &mdash; надійний захист і ком...,Вироби санітарно-гігієнічні одноразового викор...
1770,1016589,"Магнитная игра DoDo Дорожные приключения, 21.5...",295.0,295.0,https://eva.ua/pr1276718/,Аксессуары по уходу за малышами,Игрушки для малышей,"Ігри,пазли",ДоДо,Магнітна креативна гра Дорожні пригоди знайоми...,
3570,1022771,"Жиросжигатель Genius Nutrition Lipo X5, 60 капсул",1242.0,856.0,https://eva.ua/pr1301570/,"Диета, питание и витамины",Биологически активные добавки,Средства для контроля массы тела,Жениус Нутришн,Дiєтична дoбaвка Genius Nutrition Lipo X5 може...,"Приймати по 1-2 капсули на день, запиваючи вел..."


In [19]:
# sanity_query_chroma.py
import os
from openai import OpenAI
import chromadb

# === Параметры (подгони при необходимости)
CHROMA_PATH   = os.getenv("CHROMA_PATH", "./chroma_data")
CHANNEL       = os.getenv("VECTOR_NAME", "title")  # title|desc|usage|category
COLL_NAME     = f"eva_products_{CHANNEL}"
MODEL_EMB     = os.getenv("MODEL_EMB", "text-embedding-3-small")  # 1536-dim

# === Клиенты
oa = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
chroma = chromadb.PersistentClient(path=CHROMA_PATH)
col = chroma.get_collection(COLL_NAME)  # упадёт, если коллекции нет — это полезно

try:
    count = col.count()  # доступно в chromadb 0.5+
except Exception:
    count = None
print(f"Connected to collection: {COLL_NAME} (count={count})")

# === Запрос
q = "Гель-мусс для бровей "
q_vec = oa.embeddings.create(model=MODEL_EMB, input=[q]).data[0].embedding  # 1536d

# where = {"pcat": "Парфуми"}   # раскомментируй, если хочешь отфильтровать категорию
where = None

res = col.query(
    query_embeddings=[q_vec],
    n_results=5,
    include=["metadatas", "distances"],  # <-- без "ids"
    where=where
)

ids       = res.get("ids", [[]])[0]
metas     = res.get("metadatas", [[]])[0]
distances = res.get("distances", [[]])[0]
print(metas)

if not ids:
    print("Нічого не знайдено.")
else:
    for i, (rid, meta, d) in enumerate(zip(ids, metas, distances), start=1):
        sim = 1.0 - float(d) if d is not None else None
        sim_s = f"{sim:.3f}" if sim is not None else "—"
        print(f"\n#{i}  id={rid}  similarity={sim_s}")

        # Печатаем ВСЮ мета-инфу
        # meta содержит ключи: price, url, pgr, pgr1, text, category_path, sku, spec_price, on_sale, brand, pgr2, pcat
        for k in sorted(meta.keys()):
            v = meta[k]
            if isinstance(v, float):
                # аккуратное форматирование цен/чисел
                v = f"{v:.4f}".rstrip("0").rstrip(".")
            print(f"  {k}: {v}")

Connected to collection: eva_products_title (count=5000)
[{'on_sale': True, 'pgr': 'Декоративная косметика для глаз и бровей', 'pgr2': 'Мэйбелин', 'text': 'гель-мусс для бровей maybelline new york superfluff brow mousse 262 black brown, 5 мл', 'pgr1': 'Контуры для век и бровей', 'spec_price': 368.0, 'sku': '1028673', 'url': 'https://eva.ua/pr1292402/#/355802/', 'pcat': 'Косметика декоративная', 'price': 460.0, 'category_path': 'косметика декоративная > декоративная косметика для глаз и бровей > контуры для век и бровей > мэйбелин', 'brand': 'мэйбелин'}, {'pgr2': 'Мэйбелин', 'category_path': 'косметика декоративная > декоративная косметика для глаз и бровей > контуры для век и бровей > мэйбелин', 'brand': 'мэйбелин', 'on_sale': True, 'sku': '1028672', 'spec_price': 368.0, 'price': 460.0, 'pcat': 'Косметика декоративная', 'url': 'https://eva.ua/pr1292402/#/355796/', 'pgr': 'Декоративная косметика для глаз и бровей', 'pgr1': 'Контуры для век и бровей', 'text': 'гель-мусс для бровей maybel

In [1]:
import os

DIR = "./data/parfumes_knowledge_db"

for name in os.listdir(DIR):
    if name.endswith(".txt"):
        with open(os.path.join(DIR, name), "r", encoding="utf-8") as f:
            content = f.read()
        print(content)

SKU: 582513
Назва: Moschino Toy Boy Парфумована вода чоловіча, 50 мл
Бренд: Moschino
Серія: Toy
Країна: Італія
Ціна: 1420.0
URL: https://eva.ua/ua/pr174068/#/73298/
Посилання на фото: https://pwa-api.eva.ua/img/512/512/resize/5/8/582513_1_1745329392.jpg
Стать: Чоловіча
Основниі акорди: Трояндовий, Квітковий, Мускусний
Класифікація аромату: Деревні, Пряні, Фужерні
Клас: Елітна
Початкова нота: Бергамот, Груша, Мускатний горіх, Рожевий перець, Елемі
Кінцева нота: Амбра, Ветивер, Кашмеран, Сандал, Sylkolide
Тип застосування: Вечір, День, Зима, Осінь
Нота серця: Гвоздика, Магнолія, Льон, Троянда

SKU: 489636
Назва: Trussardi Delicate Rose Туалетна вода жіноча, 100 мл
Бренд: Trussardi
Серія: 0
Країна: Італія
Ціна: 1770.0
URL: https://eva.ua/ua/pr22082/#/73256/
Посилання на фото: https://pwa-api.eva.ua/img/512/512/resize/4/8/489636_1_1738144706.jpg
Стать: Жіноча
Основниі акорди: Деревний, Зелений, Озоновий, Трояндовий, Свіжий, Фруктовий, Квітковий, Цитрусовий, Мускусний, Акватичний
Класифікац